# Text Processing

This notebook processes OCR-extracted text from Nordic travel literature CSV files through a comprehensive pipeline:

## Overview
- **Input**: Raw CSV files from `literature csv_raw` folder containing OCR-extracted text
- **Output**: Cleaned CSV files in `literature csv_clean` folder with corrected text
- **Purpose**: Prepare historical travel literature text for NLP analysis

## Processing Pipeline
1. **Text Preprocessing**: Cleans and normalizes OCR text by removing soft hyphens, standardizing punctuation, and collapsing whitespace
2. **OCR Correction**: Applies rule-based corrections using a dictionary of common OCR errors while preserving original capitalization
3. **Batch Processing**: Processes all CSV files in the input folder with progress tracking
4. **Output Generation**: Creates new columns with preprocessed content, corrected text, and correction tracking data

## Key Features
- Preserves original text alongside corrected versions
- Tracks all corrections made with context
- Maintains original capitalization patterns
- Provides progress bars for batch processing
- Handles large datasets efficiently

In [27]:
# Data manipulation and file operations
import pandas as pd  # For creating and managing DataFrames
import os  # For file and directory operations
import re # For handling text processing patterns

# Progress tracking
from tqdm.notebook import tqdm  # Display progress bars in Jupyter notebooks
tqdm.pandas()

## Text Preprocessing Function

This script cleans and normalizes text by:
- Removing soft hyphens and line breaks
- Standardizing punctuation (spaces around `.`, `,`, `!`, `?`)
- Removing special characters while preserving basic punctuation
- Optionally removing numbers and words containing numbers
- Collapsing multiple whitespaces into single spaces

Use this function to prepare OCR-extracted text for NLP analysis.


In [1]:
# preprocess text
def preprocess_text(text_string):
    # Avoid data type errors
    _text_string = str(text_string)
    # Store text data in variable
    # Remove newlines, replace apostrophe s with s, remove quotation marks
    text_string_replace = _text_string.replace(r' \xad\n', '').replace(r'\xad\n', '') \
        .replace(r'\xad', '').replace(r'\xad ', '') \
        .replace(r'-\n', '').replace('—', ' ').replace(r'\n', ' ') \
        .replace("'s", "s").replace('"', ' ').replace("'", " ") \
        .replace('.', ' . ').replace(' .', ' . ').replace(',', ' ,') \
        .replace('!', ' !').replace('?', ' ?')

    # scrub for multiple signs
    # but keep full stops, commas, exclamation, and question marks
    text_string_re = re.sub(r':|;|\||\+|\"|‘|’|“|”|\’|…|\-|–|—|\$|&|\*|>|<|\/|»|«', '', text_string_replace)

    # Remove "standalone" numbers, but keep numbers that are part of words (e.g. "20th") (optional)
    text_string_re = re.sub(r'\b\d+\b', ' ', text_string_re)
    
    # Remove multiple whitespaces
    text_string_sub_additional_white_space = re.sub(r'\s+', ' ', text_string_re)

    return text_string_sub_additional_white_space

## OCR Correction Function


This function corrects common OCR errors in text by:
- Loading correction rules from `Underwoods_CorrectionRules.txt` file. (Source: [CorrectionRules.txt](https://github.com/tedunderwood/DataMunging/blob/master/rulesets/CorrectionRules.txt)
- Creating a dictionary mapping common OCR errors to correct words
- Preserving original capitalization (title case, uppercase, lowercase)
- Tracking corrections made and providing context around corrected words
- Returning both the corrected text and a detailed corrections tracker

Use this function to improve OCR-extracted text quality for NLP analysis.



In [2]:
# Read the correction rules from the file
with open('Underwoods_CorrectionRules.txt', 'r', encoding='utf-8') as f:
    CorrectionRules_string = f.read()

# Process the correction rules
CorrectionRules_list = CorrectionRules_string.split('\n')
list_of_CorrectionRules_list = [i.split('\t') for i in CorrectionRules_list]
better_list_of_CorrectionRules = [i[0:2] for i in list_of_CorrectionRules_list]

# Add additional pairs
observations_to_add = [["Bruffels", 'Brussels'], ["fix", 'six'], ['Ruffia', 'Russia']]
better_list_of_CorrectionRules.extend(observations_to_add)

# Build correction dictionary
correction_dict = dict(better_list_of_CorrectionRules)


# OCR Correction Function
def ocr_correction(text):
    # Prepare for correction
    ### Keep track of the conditions of the words (capital, title, upper,lower)
    tokens = text.split()

    # Track the original case of each word
    original_case = [i for i in tokens]
    clean_text_list = [i.lower() for i in tokens]

    # Initialize the list to store corrected words and a dictionary to track corrections
    new_word_list = []
    corrections_tracker = {}

    # Iterate through the clean_text_list word list
    for index, word in enumerate(clean_text_list):
        # Check if the word has a correction in the dictionary
        if word in correction_dict:
            # If it does, append the corrected word to the new list and track the correction
            corrected_word = correction_dict[word]
            # Restore the original case of the word
            if original_case[index].istitle():
                corrected_word = corrected_word.capitalize()
            elif original_case[index].isupper():
                corrected_word = corrected_word.upper()
            new_word_list.append(corrected_word)
            corrections_tracker[word] = corrected_word

            # Extract context around the corrected word
            start_index = max(0, index - 10)
            end_index = min(len(clean_text_list), index + 11)
            context = clean_text_list[start_index:end_index]

            # Store the context with the correction
            corrections_tracker[word] = {
                'corrected_word': corrected_word,
                'context': ' '.join(context)
            }
        else:
            # If it doesn't, append the original word
            new_word_list.append(original_case[index])

    # Send data back to text
    new_corrected_text = ' '.join(new_word_list)

    return corrections_tracker, new_corrected_text

In [30]:
# ============================================
# BATCH PROCESSING: Process all csv files
# ============================================

# Define input and output directories
departure_folder = r'.\literature csv_raw'     # Source folder containing CSV files
arrival_folder = r'.\literature csv_clean'    # Destination folder for CSV output

# Create the output directory if it doesn't exist
# exist_ok=True prevents errors if the directory already exists
os.makedirs(arrival_folder, exist_ok=True)

# Get a list of all files in the CSV folder
files_in_folder = os.listdir(departure_folder)

# Filter to only process CSV files and skip hidden files/folders
csv_files = [file for file in files_in_folder if file.endswith('.csv') and not file.startswith('.')]

# Process each file in the folder with a progress bar
for file in tqdm(csv_files, desc='Text processing', colour='blue'):

    print (f'Starting on {file}\n')
    
    # Construct the full path to the CSV file
    file_path = os.path.join(departure_folder, file)

    # Read the CSV into a Dataframe    
    df = pd.read_csv(file_path)

    # Apply the Preprocess Text function and add a new column holding the preprocessed content
    df['preprocessed_content'] = df['page_content'].progress_apply(lambda x : preprocess_text(x))

     # Apply the OCR Correction function and make two new columns and add them to the dataframe
    df['corrections_tracker'], df['ocr_corrected_text'] = zip(*tqdm(df['preprocessed_content'].apply(ocr_post_correction), desc="Applying OCR post-correction"))

    # Building a new dataframe
    df = df[['source', 'page_content', 'preprocessed_content', 'ocr_corrected_text', 'corrections_tracker']]

    # Construct the full path for the output CSV file
    csv_file_path = os.path.join(arrival_folder, file)

    # Save the DataFrame to CSV
    # index=False prevents pandas from writing row numbers as a column
    df.to_csv(csv_file_path, index=False)

    print (f'Done with {file}\n')

print (f'Done. Find your files in {arrival_folder}')

Text processing:   0%|          | 0/159 [00:00<?, ?it/s]

Starting on 1772 Travels through Holland Flanders Germany Denmark Sweden Lapland Russia the Ukraine and Poland in the years 1768 1769 and 1770 Vol 1.csv



  0%|          | 0/202 [00:00<?, ?it/s]

Applying OCR post-correction:   0%|          | 0/202 [00:00<?, ?it/s]

Done with 1772 Travels through Holland Flanders Germany Denmark Sweden Lapland Russia the Ukraine and Poland in the years 1768 1769 and 1770 Vol 1.csv

Starting on 1773 Travels through Holland Flanders Germany Denmark Sweden Lapland Russia the Ukraine and Poland in the years 1768 1769 and 1770 Vol 3.csv



...

Applying OCR post-correction:   0%|          | 0/1494 [00:00<?, ?it/s]

Done with 1918 The Charm of Scandinavia.csv

Starting on 1921 Denmark.csv



  0%|          | 0/309 [00:00<?, ?it/s]

Applying OCR post-correction:   0%|          | 0/309 [00:00<?, ?it/s]

Done with 1921 Denmark.csv

Done. Find your files in .\literature csv_clean
